In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/news.csv


In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [89]:
data=pd.read_csv('/kaggle/input/fake-news/news.csv')
data.shape

(6335, 4)

In [90]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [92]:
data.drop('Unnamed: 0',axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 148.6+ KB


In [93]:
Y=data['label']
Y[Y=='FAKE']=0
Y[Y=='REAL']=1

X=data.drop('label',axis=1)
print(X.shape,Y.shape)

(6335, 2) (6335,)


# **Data Preprocessing**

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [95]:
X,X_test,Y,Y_test=train_test_split(X,list(Y),test_size=0.2,random_state=29)
print(len(X),len(Y))
print(len(X_test),len(Y_test))

5068 5068
1267 1267


## Remove Stop Words and Vectorize using TFIDF

In [96]:
TitleTFIDF=TfidfVectorizer(stop_words='english')
TextTFIDF=TfidfVectorizer(stop_words='english')

TitleTFIDF.fit(X['title'])
TextTFIDF.fit(X['text'])

X_trainTitle=TitleTFIDF.transform(X['title'])
X_trainText=TextTFIDF.transform(X['text'])

X_testTitle=TitleTFIDF.transform(X_test['title'])
X_testText=TextTFIDF.transform(X_test['text'])

X_train=hstack((X_trainTitle,X_trainText))
X_test=hstack((X_testTitle,X_testText))

In [97]:
print(X_train.shape,X_test.shape)

(5068, 69735) (1267, 69735)


In [98]:
print(len(TitleTFIDF.get_feature_names()))
print(len(TextTFIDF.get_feature_names()))

8772
60963


## Models

In [99]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

In [100]:
model=SGDClassifier(random_state=29)

cv=GridSearchCV(model,param_grid={'eta0' : [0.0001,0.001,0.01,0.1,1],'alpha' : [0.0001,0.001,0.01,0.1]})

cv.fit(X_train,list(Y))

GridSearchCV(estimator=SGDClassifier(random_state=29),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'eta0': [0.0001, 0.001, 0.01, 0.1, 1]})

In [101]:
cv.best_params_

{'alpha': 0.0001, 'eta0': 0.0001}

In [102]:
cv.best_score_

0.9279790728420085

In [104]:
model=SGDClassifier(alpha=0.001,eta0=0.0001)
model.fit(X_train,list(Y))
print('Train Accuracy : ',model.score(X_train,list(Y)))
print('Test Accuracy : ',model.score(X_test,list(Y_test)))

Train Accuracy :  0.9704025256511445
Test Accuracy :  0.9131807419100236


# Naive Bayes

In [106]:
model=MultinomialNB()
cv=GridSearchCV(model,param_grid={'alpha' : [0.00001,0.0001,0.001,0.01,0.1,1]})
cv.fit(X_train,list(Y))

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]})

In [107]:
cv.best_params_

{'alpha': 0.1}

In [108]:
cv.best_score_

0.9013393926295438

In [111]:
model=MultinomialNB(alpha=0.1)
model.fit(X_train,list(Y))

print('Train Accuracy : ',model.score(X_train,list(Y)))
print('Test Accuracy : ',model.score(X_test,list(Y_test)))


AttributeError: list not found